# Загрузка библиотек

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import random
import networkx as nx
import pandas as pd
from itertools import combinations

игнорированеи внутренних системных ошибок, и игнорирование всплывающих предупреждений

In [ ]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

import warnings
warnings.filterwarnings("ignore")

Инициализация двух моделей, для обучения используется модель AdvancedDroneNet, так как более ее простая версия не показывала результатов. Линейный слой (nn.Linear) с 4 входами (координаты x, y и скорости vx, vy) и 128 выходами, за которым следует функция активации ReLU. Ещё один линейный слой, увеличивающий размерность с 128 до 256, также с функцией активации ReLU. Слой дропаута (nn.Dropout) с вероятностью отключения каждого нейрона 0.5 для регуляризации и предотвращения переобучения. Следующий линейный слой уменьшает размерность с 256 обратно до 128, снова применяется функция активации ReLU. Завершающий линейный слой с 128 входами и 4 выходами, предполагается, что это количество возможных действий или предсказаний модели. Метод forward(x) принимает входные данные x и пропускает их через определённую последовательность слоёв, возвращая выходные данные модели. 

In [ ]:
class DroneNet(nn.Module):
    def __init__(self):
        super(DroneNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 128),  # Соответствие входных данных: x, y, vx, vy
            nn.ReLU(),
            nn.Linear(128, 4)   # 4 возможных действия для выхода
        )

    def forward(self, x):
        return self.fc(x)

In [ ]:
class AdvancedDroneNet(nn.Module):
    def __init__(self):
        super(AdvancedDroneNet, self).__init__()
        self.fc = nn.Sequential(
            nn.Linear(4, 128),  # Входные данные: x, y, vx, vy
            nn.ReLU(),
            nn.Linear(128, 256), # доп слой
            nn.ReLU(),
            nn.Dropout(0.5),     # дропаут для регуляризации
            nn.Linear(256, 128), # уменьшение размерность
            nn.ReLU(),
            # убран BatchNorm1d , приводил к ошибкам
            nn.Linear(128, 4)    # 4 возможных действия для выхода
        )

    def forward(self, x):
        return self.fc(x)

# Объект дрона, обучающая часть (ООП)

Этот код определяет класс Drone, который моделирует поведение дрона, включая его перемещение, выбор действий и обучение с использованием нейронной сети. 

* Инициализация

При создании экземпляра дрона задаются его идентификатор (для образования некой иерархии), путь к модели нейронной сети, начальные координаты и скорость, очки, инициализируется нейронная сеть и оптимизатор (Adam), а также устанавливается параметр epsilon для эпсилон-жадной стратегии выбора действий. Модель нейронной сети загружается из указанного пути, если он предоставлен.

* Выбор действия

Метод select_action использует текущее состояние дрона для выбора следующего действия. С некоторой вероятностью epsilon действие выбирается случайно (исследование), в остальных случаях действие выбирается на основе предсказаний нейронной сети (использование политики), максимизирующее оценку действия.

* Перемещение

Метод move обновляет положение и скорость дрона на основе выбранного действия и преобразования этого действия в изменение ускорения. Положение и скорость дрона ограничиваются заданными пределами, чтобы дрон не выходил за границы заданного пространства.

* Преобразование действия в ускорение 

Метод action_to_acceleration определяет, как конкретное действие изменяет ускорение дрона.

* Логирование действий

Метод log_action выводит информацию о текущем действии дрона, его позиции, скорости и набранных очках.

* Сохранение и загрузка модели

Методы save_model и load_model позволяют сохранять состояние нейронной сети дрона в файл и загружать его соответственно. Это обеспечивает возможность продолжения обучения или использования обученной модели без необходимости повторного обучения с нуля.

Класс предоставляет полный функционал для моделирования поведения дрона, включая обучение с помощью обучения с подкреплением, выбор действий в зависимости от текущего состояния и возможность сохранения/загрузки обученной модели.

In [ ]:
class Drone:
    def __init__(self, drone_id, model_path, x=0, y=0, vx=0, vy=0):
        self.drone_id = drone_id
        self.position = np.array([x, y], dtype=np.float32) # начальная позиция [x, y]
        self.velocity = np.array([vx, vy], dtype=np.float32) # начальная скорость [Vx, Vy]
        self.points = 0  # Начальное количество очков
        self.network = AdvancedDroneNet()
        self.optimizer = optim.Adam(self.network.parameters(), lr=0.01)
        self.epsilon = 0.1  # Для эпсилон-жадной стратегии
        self.load_model(model_path)
    def select_action(self, state):
        if np.random.rand() < self.epsilon:  # Исследование
            return np.random.choice([0, 1, 2, 3])  # Случайный выбор действия, чтобы появились
            #случайные блуждания при обучении. 
        else:  # Использование политики
            state_tensor = torch.FloatTensor(state).unsqueeze(0)  # преобразование состояния в тензор
            action_values = self.network(state_tensor)
            return torch.argmax(action_values).item()  # выбор действия с максимальной оценкой
        
    def move(self):
        state = np.concatenate((self.position, self.velocity))
        action = self.select_action(state)
        # преобразование действия в изменение скорости
        acceleration_change = self.action_to_acceleration(action)
        self.velocity += acceleration_change
        self.position += self.velocity
        
        # ограничение движения в пределах заданного пространства
        self.position = np.clip(self.position, -40, 40)
        self.velocity *= np.where((self.position == 40) | (self.position == -40), -1, 1)
        
    def action_to_acceleration(self, action):
        if action == 0: return np.array([0.1, 0])  # Ускорение вправо
        elif action == 1: return np.array([-0.1, 0])  # Ускорение влево
        elif action == 2: return np.array([0, 0.1])  # Ускорение вверх
        elif action == 3: return np.array([0, -0.1])  # Ускорение вниз
        return np.array([0, 0])

    def log_action(self, action):
        print(f"Drone {self.drone_id}: {action}, Позиция: {self.position}, Скорость: {self.velocity}, Очки: {self.points}")
    
    def save_model(self, file_name):
        torch.save(self.network.state_dict(), file_name)
        print(f"Модель сохранена в {file_name}")

    def load_model(self, file_name):
        if file_name == 'none':
            pass
        else:
            self.network.load_state_dict(torch.load(file_name))
            self.network.eval()  # переключение в режим оценки
            print(f"Модель загружена из {file_name}")

# Объект группы дронов, обучающая часть (ООП)

Код определяет класс DroneGroup, который управляет группой дронов, позволяя добавлять дроны в группу, проверять столкновения между ними, симулировать их движение и визуализировать их позиции. 

* Инициализация

При создании экземпляра DroneGroup, он инициализируется с пустым списком дронов.

* Добавление дрона

Метод add_drone добавляет дрон в группу.

* Проверка столкновений 

Метод check_collisions проходит по всем парам дронов в группе, вычисляет расстояние между ними и в случае, если расстояние меньше 2 (порог столкновения), уменьшает количество очков каждого дрона на 10 и логирует факт столкновения.

* Симуляция

Метод simulate выполняет симуляцию движения всех дронов в группе на заданное количество шагов (steps). На каждом шаге каждый дрон перемещается, ему начисляются очки за успешное перемещение, логируется изменение его позиции, после чего проверяются столкновения. Также на каждом шаге определяется и логируется дрон с наибольшим количеством очков и визуализируются позиции всех дронов.

*Поиск лучшего дрона

Метод find_best_drone идентифицирует дрона с наибольшим количеством очков в группе и выводит его идентификатор и количество очков.

*Визуализация позиций

Метод plot_group_positions создает график, на котором отображаются текущие позиции всех дронов в группе, используя библиотеку matplotlib для отрисовки.

Этот класс обеспечивает комплексные средства для моделирования взаимодействий между группой дронов, их движения, обработки столкновений, а также визуального представления их позиций в пространстве.

In [ ]:
class DroneGroup:
    def __init__(self):
        self.drones = [] # Инициализация списка дронов
    
    def add_drone(self, drone):
        self.drones.append(drone) # Добавление дронов в группу (список)

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], start=i+1):
                distance = np.linalg.norm(drone1.position - drone2.position)
                if distance < 2:  # Порог столкновения
                    drone1.points -= 10
                    drone2.points -= 10
                    drone1.log_action(f"Столкновение с Drone {drone2.drone_id}")
                    drone2.log_action(f"Столкновение с Drone {drone1.drone_id}")

    def simulate(self, steps=10):
        for step in range(steps):
            print(f"| ЭПОХА {step + 1} |")
            for drone in self.drones:
                drone.move()
                drone.points += 1  # Начисление очков за успешный шаг без столкновения
                drone.log_action("Изменение позиции")
            self.check_collisions()  # Проверка и обработка столкновений
            self.find_best_drone()
            self.plot_group_positions()
    def find_best_drone(self):
        best_drone = None
        max_points = -float('inf')  # Инициализируем с очень маленьким числом
        count_point = 0
        print('<=======>')
        for drone in self.drones:
            if drone.points > max_points:
                count_point +=1
                best_drone = drone
                max_points = drone.points
        print(f"Лучший дрон: Drone {best_drone.drone_id} с очками: {max_points}")
        print('<=======>')      
    def plot_group_positions(self):
        plt.figure(figsize=(10, 6))
        for drone in self.drones:
            plt.scatter(drone.position[0], drone.position[1], label=f'Drone {drone.drone_id}')
        plt.xlim(-50, 50)
        plt.ylim(-50, 50)
        plt.xlabel('X позиция')
        plt.ylabel('Y позиция')
        plt.title('Позиции дронов')
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), shadow=True, ncol=8)
        plt.show()

Запуск симуляции группы дронов в количестве 24 штук. Изначально генерируются в случайных позициях и со случайными скоростями. После чего, обучаютсяня на протяжении 500 шагов (эпох)

In [ ]:
drones_group = DroneGroup()
for i in range(24):  # Добавление дронов
    drones_group.add_drone(Drone(drone_id=i, model_path='none',x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                 vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))
drones_group.simulate(steps=500)

После просшествия 500 эпох и статистике, находим дрона с наибольшим количеством очков. Это означает, что такая модель столкнулась с другими дронами наименьшее количество раз. Эта задача и стояла в обучении. Таким образом, мы получили экземпляр дрона, который научился игнорировать сталкивания с другими объектами. Обученную модель такого дрона мы сохраняем в файл с весами модели, для последующего использования

In [ ]:
drone = drones_group.drones[14]  
drone.save_model("circle_1_best_drone14.pth")

Используем ранее обученную модель в следствии ансамблевого обучения. Теперь количество эпох кратно уменьшается, и мы получаем симуляцию действий уже у обученных моделей. Наилучший результат - 210, это означает, что за 250 шагов дрон столкнулся лишь 4 раза. Важно заметить, что у остальных дронов количество очков тоже стабилизовалось, и у большего преимущества они находятся в положительном достатке, что означает, что модель научилась игнорировать сталкивания еще лучше. Фактор случайного действия в обучении такого рода модели просто не обходим - так как датчики подвижны, они могут быть подвержены внешним стихийным или нет воздействиям. Таким образом, случайные ускорения в ту или иную сторону могут являтся дуновением ветра или что-то вроде того.

In [ ]:
model_path = "circle_1_best_drone14.pth" 
drones_master = DroneGroup()
for i in range(24):
    drones_master.add_drone(Drone(drone_id=i, model_path=model_path, x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                 vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))
drones_master.simulate(steps=250)

In [ ]:
drone = drones_master.drones[23]  
drone.save_model("circle_2_best_drone23.pth")

На этом этапе делаем третий цикл обучения самых лучших весов предыдущей модели, чтобы проверить, можно ли обучить дронов еще лучше. Как оказалось - можно.

In [ ]:
model_path = "circle_2_best_drone23.pth" 
drones_master_up = DroneGroup()
for i in range(24):
    drones_master_up.add_drone(Drone(drone_id=i, model_path=model_path, x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                 vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))
drones_master_up.simulate(steps=250)

In [ ]:
drone = drones_master.drones[23]  
drone.save_model("circle_3_best_drone23.pth")

In [ ]:
model_path = "circle_3_best_drone23.pth" 
drones_master_up_test = DroneGroup()
for i in range(24):
    drones_master_up_test.add_drone(Drone(drone_id=i, model_path=model_path, x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                 vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))
drones_master_up_test.simulate(steps=100)

Здесь мы достигли переобучения, а значит, следующую модель мы сохраним просто так. Обучение получилось в следствии того, что модель достигла наилучшего результата в задаче не сталкиваться, и начала умышленно группировать дроны так, чтобы образовывались столкновения. Иными словами, она начала учиться получать максимальное количество очков. Такое не нужно, поэтому в дальнейшейм исследовании используем веса предидущей модели.

In [ ]:
drone = drones_master_up_test.drones[1]  
drone.save_model("circle_4_best_drone1.pth")

Этот код определяет класс Graph, который реализует базовую структуру данных графа для хранения связей между узлами (вершинами), такими как дроны, с использованием словаря. 

* Инициализация 

Создает экземпляр графа с пустым словарем edges для хранения связей между вершинами.

* Добавление ребра

Метод add_edge добавляет ребро между двумя узлами (node1 и node2) с заданным весом weight. Если узлы еще не существуют в графе, они добавляются в словарь edges, а их связь определяется весом weight. Ребро является двунаправленным, то есть добавляется как для node1 -> node2, так и для node2 -> node1.

* Удаление ребра

Метод remove_edge удаляет ребро между двумя узлами. Если узлы и их связь существуют в графе, они удаляются из словаря edges.

* Получение соседей

Метод get_neighbors возвращает всех соседей (связанные узлы) для заданного узла node. Если узел присутствует в графе, метод возвращает словарь его соседей и соответствующих весов ребер; если узел отсутствует, возвращается пустой словарь.

Класс Graph предоставляет удобный интерфейс для работы с графами, позволяя легко добавлять, удалять связи между узлами и получать информацию о соседях узла, мы будем использовать в дальнейшем для моделирования марширутов.

In [ ]:
class Graph:
    def __init__(self):
        self.edges = {}  # Словарь для хранения связей между вершинами (дронами)

    def add_edge(self, node1, node2, weight=1):
        # Добавляет ребро между node1 и node2 с весом weight
        if node1 in self.edges:
            self.edges[node1][node2] = weight
        else:
            self.edges[node1] = {node2: weight}
        
        if node2 in self.edges:
            self.edges[node2][node1] = weight
        else:
            self.edges[node2] = {node1: weight}

    def remove_edge(self, node1, node2):
        # Удаляет ребро между node1 и node2
        if node1 in self.edges:
            del self.edges[node1][node2]
        if node2 in self.edges:
            del self.edges[node2][node1]

    def get_neighbors(self, node):
        # Возвращает соседей для данного узла
        return self.edges.get(node, {})

# Объект группы дронов, тестовая часть (ООП)

Модернизация ранее использованного класса DroneGroup. 
* Обновление графа во время симуляции

В метод simulate добавлены вызовы нового метода update_graph до и после выполнения шагов симуляции. Этот метод обновляет граф взаимодействий между дронами, что позволяет отслеживать изменения в их взаимоотношениях на основе их текущих позиций.

* Метод update_graph

Этот новый метод создает или пересоздает граф взаимодействий между дронами, используя текущие позиции дронов для определения их связей. Связи в графе создаются, если расстояние между дронами меньше заданного порога (10 в данном случае), что указывает на потенциальное взаимодействие или близость дронов.

* Визуализация графа

Добавлен метод plot_graph, который использует библиотеку NetworkX для визуализации обновленного графа взаимодействий между дронами. В этом методе узлы представляют дроны, рёбра указывают на их взаимодействие, а вес ребра отображает расстояние между дронами. Метод применяет макет Kamada-Kawai для расположения узлов и визуализирует узлы и рёбра с учетом их атрибутов, таких как вес ребра и степень узла.

Эти изменения делают класс DroneGroup более мощным инструментом для моделирования и визуализации динамических взаимодействий и отношений в группе дронов. Обновление графа взаимодействий в реальном времени во время симуляции предоставляет более глубокое понимание структуры и динамики группы дронов, а визуализация этих взаимодействий через граф дает наглядное представление об этих отношениях.

In [ ]:
class DroneGroup:
    def __init__(self):
        self.drones = []
    
    def add_drone(self, drone):
        self.drones.append(drone)

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], start=i+1):
                distance = np.linalg.norm(drone1.position - drone2.position)
                if distance < 2:  # Порог столкновения
                    drone1.points -= 10
                    drone2.points -= 10
                    drone1.log_action(f"Столкновение с Drone {drone2.drone_id}")
                    drone2.log_action(f"Столкновение с Drone {drone1.drone_id}")

    def simulate(self, steps=10):
        for step in range(steps):
            print(f"| ЭПОХА {step + 1} |")
            self.update_graph()
            for drone in self.drones:
                drone.move()
                drone.points += 1  # Начисление очков за успешный шаг без столкновения
                drone.log_action("Изменение позиции")
            self.check_collisions()  # Проверка и обработка столкновений
            self.find_best_drone()
            self.plot_group_positions()
            # После выполнения некоторых шагов симуляции
            self.update_graph()  # Обновляем граф на основе текущих позиций дронов
            self.plot_graph()  # Визуализируем граф

    def find_best_drone(self):
        best_drone = None
        max_points = -float('inf')  # Инициализируем с очень маленьким числом
        count_point = 0
        print('<=======>')
        for drone in self.drones:
            if drone.points > max_points:
                count_point +=1
                best_drone = drone
                max_points = drone.points
        print(f"Лучший дрон: Drone {best_drone.drone_id} с очками: {max_points}")
        print('<=======>')
    def update_graph(self):
        # Обновляет граф на основе текущих позиций дронов
        self.graph = Graph()  # Пересоздаем граф для обновления связей
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], start=i+1):
                distance = np.linalg.norm(drone1.position - drone2.position)
                if distance < 10:  # Условный порог для создания связи
                    self.graph.add_edge(drone1.drone_id, drone2.drone_id, weight=distance)
    def plot_group_positions(self):
        plt.figure(figsize=(10, 6))
        for drone in self.drones:
            plt.scatter(drone.position[0], drone.position[1], label=f'Drone {drone.drone_id}')
        plt.xlim(-50, 50)
        plt.ylim(-50, 50)
        plt.xlabel('X позиция')
        plt.ylabel('Y позиция')
        plt.title('Позиции дронов')
        plt.legend(loc='upper center', bbox_to_anchor=(0.5, -0.1), shadow=True, ncol=8)
        plt.show()
    def plot_graph(self):
        G = nx.Graph()
        plt.figure(figsize=(10, 4))
        # Добавляем узлы и рёбра в граф NetworkX
        for drone_id, neighbors in self.graph.edges.items():
            G.add_node(drone_id)
            for neighbor_id, weight in neighbors.items():
                G.add_edge(drone_id, neighbor_id, weight=weight)

        # Используем макет Kamada-Kawai для расположения узлов
        pos = nx.kamada_kawai_layout(G)

        # Размеры узлов основаны на степени узла (количество связей)
        node_sizes = [50 * G.degree(n) for n in G.nodes]

        # Цвета рёбер основаны на их весе
        edge_colors = [G[u][v]['weight'] for u, v in G.edges]

        # Визуализация узлов
        nx.draw_networkx_nodes(G, pos, node_size=node_sizes, cmap=plt.cm.plasma)

        # Визуализация рёбер
        nx.draw_networkx_edges(G, pos, width=2, edge_color=edge_colors, edge_cmap=plt.cm.Blues)

        # Визуализация меток узлов
        nx.draw_networkx_labels(G, pos)
        
        plt.xlim(-1.5, 1.5)
        plt.ylim(-1.5, 1.5)

        plt.title("Граф взаимодействия дронов")
        plt.colorbar(plt.cm.ScalarMappable(norm=plt.Normalize(vmin=min(edge_colors), vmax=max(edge_colors)), cmap=plt.cm.Blues), label='Вес ребра')
        plt.axis('off')  # выключаем отображение осей для чистоты визуализации
        plt.show()

In [ ]:
model_path = "circle_3_best_drone23.pth" 
drone_graph = DroneGroup()
for i in range(24):
    drone_graph.add_drone(Drone(drone_id=i, model_path=model_path, x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                 vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))
drone_graph.simulate(steps=100)

In [ ]:
drone = drone_graph.drones[2]  
drone.save_model("graph_circle_5_best_drone2.pth")

После обучения модели с использованием графов, мы можем внедрить в объект дрона марширут. В данном случае для планирования марширутам мы будем использовать алгоритм А-звезды

In [ ]:
def plan_route(graph, start, goal):
    # Используем алгоритм A* для планирования маршрута
    path = nx.astar_path(graph, start, goal)
    return path

# Объект дрона, тестовая часть (ООП)

Так же класс Drone был модернизирован, чтобы внедрить в него сетевое взаимодействие через графы.

* Добавление маршрута

В конструктор добавлен новый атрибут self.route, представляющий собой список координат (представлен в примере как [20, 25, 14, 14]), к которым дрон будет стремиться переместиться. Это нововведение позволяет задать дрону конкретный маршрут движения.

* Планирование маршрута

В класс введен новый метод update_route, который использует внешнюю функцию plan_route для планирования маршрута от текущего местоположения дрона до следующей целевой точки, используя алгоритм A* и граф, представляющий среду, в которой перемещается дрон. Этот метод обновляет self.route, исключая уже достигнутую текущую позицию.

* Адаптация движения

Добавлен метод adapt_movement, который должен адаптировать движение дрона в соответствии с текущим маршрутом и ситуацией вокруг. Однако в представленном коде отсутствует конкретная реализация изменения скорости (acceleration_change) на основе self.route.

* Изменение в методе move

В методе move временно убрано использование метода select_action для выбора случайного действия, вместо этого действие захардкодено как 2 "ускорение вверх". Это изменение отклоняется от предыдущей логики выбора действия на основе эпсилон-жадной стратегии, чтобы уменьшить количество случайных блужданий.

* Планирование маршрута и A алгоритм*

Внедрение функции plan_route вне класса Drone, использующей алгоритм A* для оптимизации маршрута между двумя точками, предоставляет дрону возможность эффективного маршрутизирования в сложной среде.

Эти модификации расширяют функциональные возможности дрона, добавляя сложность и реализм в его поведение через планирование и адаптацию маршрута. 

In [ ]:
class Drone:
    def __init__(self, drone_id, model_path, x=0, y=0, vx=0, vy=0):
        self.drone_id = drone_id
        self.position = np.array([x, y], dtype=np.float32) # начальная позиция [x, y]
        self.velocity = np.array([vx, vy], dtype=np.float32) # начальная скорость [Vx, Vy]
        self.points = 0  # Начальное количество очков
        self.network = AdvancedDroneNet()
        self.optimizer = optim.Adam(self.network.parameters(), lr=0.01)
        self.epsilon = 0.1  # Для эпсилон-жадной стратегии
        self.load_model(model_path)
        self.route = [20, 25, 14, 14] # координаты, куда дроны будут перемещаться
    def select_action(self, state):
        if np.random.rand() < self.epsilon:  # Исследование
            return np.random.choice([0, 1, 2, 3])  # Случайный выбор действия
        else:  # Использование политики
            state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Преобразование состояния в тензор
            action_values = self.network(state_tensor)
            return torch.argmax(action_values).item()  # Выбор действия с максимальной оценкой
    def update_route(self, graph):
        # Планирование маршрута до следующей целевой точки
        current_position = (self.position[0], self.position[1])
        if self.route:
            next_target = self.route[0]  # Предположим, что route уже содержит координаты целевых точек
            planned_path = plan_route(graph, current_position, next_target)
            self.route = planned_path[1:]  # Обновляем маршрут, исключая текущую позицию
    def adapt_movement(self):
        # Адаптация движения на основе текущего маршрута и ситуации вокруг
        if self.route:
            next_step = self.route[0]
            self.velocity += acceleration_change
            self.position += self.velocity
    def move(self):
        state = np.concatenate((self.position, self.velocity))
        #action = self.select_action(state)
        action = 2
        # Преобразование действия в изменение скорости
        acceleration_change = self.action_to_acceleration(action)
        self.velocity += acceleration_change
        self.position += self.velocity
        
        # Ограничение движения в пределах заданного пространства
        self.position = np.clip(self.position, -40, 40)
        self.velocity *= np.where((self.position == 40) | (self.position == -40), -1, 1)
    def action_to_acceleration(self, action):
        # Пример простой реализации
        if action == 0: return np.array([0.1, 0])  # Ускорение вправо
        elif action == 1: return np.array([-0.1, 0])  # Ускорение влево
        elif action == 2: return np.array([0, 0.1])  # Ускорение вверх
        elif action == 3: return np.array([0, -0.1])  # Ускорение вниз
        return np.array([0, 0])

    def log_action(self, action):
        print(f"Drone {self.drone_id}: {action}, Позиция: {self.position}, Скорость: {self.velocity}, Очки: {self.points}")
    
    def save_model(self, file_name):
        torch.save(self.network.state_dict(), file_name)
        print(f"Модель сохранена в {file_name}")

    def load_model(self, file_name):
        if file_name == 'none':
            pass
        else:
            self.network.load_state_dict(torch.load(file_name))
            self.network.eval()  # Переключение в режим оценки
            print(f"Модель загружена из {file_name}")

Движение дрона по заданному маршируту, вместе с коммуникациями внутри сети. Так как дроны все вместе имеют одни и те же пути, они подвержены столкновениям. Тем не менее, большая часть дронов успешно движется по заданным координатам, игнорируя столкновения.

In [ ]:
model_path = "graph_circle_5_best_drone2.pth" 
drone_graph_adapt = DroneGroup()
for i in range(24):
    drone_graph_adapt.add_drone(Drone(drone_id=i, model_path=model_path, x=np.random.uniform(-30, 30), 
                                 y=np.random.uniform(-30, 30),
                                 vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))
drone_graph_adapt.simulate(steps=100)

Как видно, обученная модель дронов стала реже сталкиваться, при этом, следуя заданному маршируту. Это можно отследить по изменениям позициям дронов.

## Новая часть

### Добавлено в класс Drone:
-  в __init__ "self.is_moving = True", который отвечает за то, двигаются ли дроны в данный момент.
- Функция move_towards_target - эта функция реализует логику движения дрона к заданной цели.
- Функция action_to_acceleration_search - эта функция определяет, как изменится скорость дрона в зависимости от выбранного действия, предназначенного для случайных блужданий при поиске заражённого участка. 
- Функция random_move - Эта функция используется для имитации случайных перемещений дрона при поиске заражённого участка.

\
Функция move_towards_target: \
Проверка условий для начала движения: Если маршрут (route) не установлен или флаг is_moving установлен в False, то функция завершается, и дрон остаётся на месте. Это предотвращает движение, если дрону не была назначена цель или если его движение было остановлено 

Вычисление направления к цели: Определяется вектор, направленный от текущего положения дрона к цели (target_position). Цель обычно берётся как первый элемент в списке rou


Проверка близости к цели: Если дрон находится в непосредственной близости от цели (например, в радиусе 1 единицы от неё), движение останавливается путём установки is_moving в False. Это предотвращает "дрожание" дрона вокруг целевой точки, когда он фактически уже достиг ц 
и.

Продолжение движения к цели: Если дрон не достиг цели, он продолжает движение. Вычисляется нормализованный вектор направления к цели, который затем умножается на скорость (в примере — 5.0). Это значение добавляется к текущему положению дрона, что обеспечивает его движение к 
ели.

Ограничение движения: Положение дрона ограничивается заданными границами (в данном случае, -100 и 100 по обеим осям), чтобы он не выходил за пределы определённ

\
Функция action_to_acceleration_search:
* Выбор действия: В зависимости от полученного параметра action, функция возвращает вектор ускорения, который изменит текущую скорость дрона в одном из четырёх направлений (вправо, влево, вверх, вниз).* 
Ускорение: Величина ускорения по выбранному направлению фиксирована и равна 5 единица

\
Функция random_move:
* Выбор случайного действия: С помощью функции np.random.choice случайным образом выбирается одно из четырёх возможных действий (0, 1, 2, 3), которое затем передаётся в функцию action_to_acceleration_search для определения вектора ускорения.
* 
Применение ускорения: Полученный вектор ускорения непосредственно применяется к текущей скорости дрона, что изменяет его положени
* 

Ограничение положения: Новое положение дрона ограничивается заданными границами (от -100 до 100 по обеим осям), чтобы он не выходил за пределы зоны поиска.м.ой зоны.

In [ ]:
class Drone:
    def __init__(self, drone_id, model_path, x=0, y=0, vx=0, vy=0, group=None):
        self.drone_id = drone_id
        self.position = np.array([x, y], dtype=np.float32) # начальная позиция [x, y]
        self.velocity = np.array([vx, vy], dtype=np.float32) # начальная скорость [Vx, Vy]
        self.points = 0  # Начальное количество очков
        self.network = AdvancedDroneNet()
        self.optimizer = optim.Adam(self.network.parameters(), lr=0.01)
        self.epsilon = 0.1  # Для эпсилон-жадной стратегии
        self.group = group  # Группа дрона: 'first' или 'second'
        self.load_model(model_path)
        self.route = [20, 25, 14, 14] # координаты, куда дроны будут перемещаться
        self.is_moving = True
    def select_action(self, state):
        if np.random.rand() < self.epsilon:  # Исследование
            return np.random.choice([0, 1, 2, 3])  # Случайный выбор действия
        else:  # Использование политики
            state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Преобразование состояния в тензор
            action_values = self.network(state_tensor)
            return torch.argmax(action_values).item()  # Выбор действия с максимальной оценкой
    def update_route(self, graph):
        # Планирование маршрута до следующей целевой точки
        current_position = (self.position[0], self.position[1])
        if self.route:
            next_target = self.route[0]  # Предположим, что route уже содержит координаты целевых точек
            planned_path = plan_route(graph, current_position, next_target)
            self.route = planned_path[1:]  # Обновляем маршрут, исключая текущую позицию
    def adapt_movement(self):
        # Адаптация движения на основе текущего маршрута и ситуации вокруг
        if self.route:
            next_step = self.route[0]
            self.velocity += acceleration_change
            self.position += self.velocity
    def move(self):
        if not self.is_moving:  # Если дрон не должен двигаться, пропускаем шаг
            return
        state = np.concatenate((self.position, self.velocity))
        #action = self.select_action(state)
        action = 2
        # Преобразование действия в изменение скорости
        acceleration_change = self.action_to_acceleration(action)
        self.velocity += acceleration_change
        self.position += self.velocity
        
        # Ограничение движения в пределах заданного пространства
        self.position = np.clip(self.position, -100, 100)
        self.velocity *= np.where((self.position == 100) | (self.position == -100), -1, 1)
    def action_to_acceleration(self, action):
        # Пример простой реализации
        if action == 0: return np.array([0.1, 0])  # Ускорение вправо
        elif action == 1: return np.array([-0.1, 0])  # Ускорение влево
        elif action == 2: return np.array([0, 0.1])  # Ускорение вверх
        elif action == 3: return np.array([0, -0.1])  # Ускорение вниз
        return np.array([0, 0])

    def log_action(self, action):
        print(f"Drone {self.drone_id}: {action}, Позиция: {self.position}, Скорость: {self.velocity}, Очки: {self.points}")
    
    def save_model(self, file_name):
        torch.save(self.network.state_dict(), file_name)
        print(f"Модель сохранена в {file_name}")

    def load_model(self, file_name):
        if file_name == 'none':
            pass
        else:
            self.network.load_state_dict(torch.load(file_name))
            self.network.eval()  # Переключение в режим оценки
            print(f"Модель загружена из {file_name}")
    def move_towards_target(self):
        if not self.route or not self.is_moving:  # Если маршрут не задан или движение остановлено
            return

        target_position = np.array(self.route[0])  # Предполагаем, что цель — первый элемент маршрута
        direction_to_target = target_position - self.position

        # Проверяем, находимся ли мы в непосредственной близости от цели (например, в радиусе 1 единицы)
        if np.linalg.norm(direction_to_target) < 1.0:
            self.is_moving = False  # Останавливаем движение дрона
            return

        # Если мы не достигли цели, продолжаем движение
        normalized_direction = direction_to_target / np.linalg.norm(direction_to_target) if np.linalg.norm(direction_to_target) else np.zeros_like(direction_to_target)
        self.velocity = normalized_direction * 5.0  # Примерная скорость, направленная к цели
        self.position += self.velocity
        self.position = np.clip(self.position, -100, 100)  # Ограничиваем движение в пределах заданного пространства
       
    #случайные блуждания для поиска зараженного участка первой группы дронов.
    def action_to_acceleration_search(self, action):
        # Пример простой реализации
        if action == 0: return np.array([5, 0])  # Ускорение вправо
        elif action == 1: return np.array([-5, 0])  # Ускорение влево
        elif action == 2: return np.array([0, 5])  # Ускорение вверх
        elif action == 3: return np.array([0, -5])  # Ускорение вниз
        return np.array([0, 0])
    def random_move(self):
        action = np.random.choice([0, 1, 2, 3])
        acceleration_change = self.action_to_acceleration_search(action)
        self.velocity = acceleration_change  # Применяем изменение скорости напрямую
        self.position += self.velocity
        self.position = np.clip(self.position, -100, 100)

### Класс InfectedArea 
представляет зараженную (или после очистки — очищенную) зону в некоторой области. Этот класс используется для моделирования зараженных участков, которые могут быть обнаружены и очищены дронами в ходе симуляции. 

center: Хранит центральную точку зараженного участка в виде массива NumPy. Эта точка используется как координаты центра зараженной зоны.
radius: Определяет радиус зараженной зоны. Радиус используется для определения границ зараженной области относительно её центра.
is_cleaned: Булевый флаг, который указывает, была ли зараженная зона очищена. По умолчанию установлен в False, что означает, что зона является зараженно

is_inside(position): Метод, который проверяет, находится ли заданная позиция (как точка в пространстве) внутри границ зараженной зоны. Это достигается путём вычисления евклидова расстояния от центра зоны до указанной позиции и сравнения этого расстояния с радиусом зоны. Если расстояние меньше или равно радиусу, метод возвращает True, указывая на то, что точка находится внутри зараженной зоны.

plot(): Метод для визуализации зараженной зоны на графике. Используется библиотека Matplotlib для отрисовки круга (plt.Circle), представляющего зараженную зону, с центром в точке center и радиусом radius. Цвет зараженной зоны изменяется в зависимости от статуса очистки: красный цвет ('red') используется для обозначения зараженной зоны, а цвет 'teal' — для очищенной. Также выводится текст ('Зараженная зона' или 'Очищенная зона'), который указывает на текущий статус зоны.й.

In [ ]:
class InfectedArea:
    def __init__(self, center, radius):
        self.center = np.array(center)  # Центр зараженного участка
        self.radius = radius  # Радиус зараженного участка
        self.is_cleaned = False
    def is_inside(self, position):
        # Проверка, находится ли заданная позиция внутри зараженного участка
        distance = np.linalg.norm(position - self.center)
        return distance <= self.radius
        
    def plot(self):
        color = 'teal' if self.is_cleaned else 'red'
        text = 'Очищенная зона' if self.is_cleaned else 'Зараженная зона'
        circle = plt.Circle((self.center[0], self.center[1]), self.radius, color=color, alpha=0.3)
        plt.gca().add_patch(circle)
        plt.text(self.center[0], self.center[1], text, color=color, fontsize=12, ha='center', va='center')


 plot_group_positions

Этот метод отвечает за визуализацию текущего расположения всех дронов на графике. Дроны разделяются по группам с помощью разных цветов

Дроны первой группы отображаются синим цветом.
Дроны второй группы отображаются зелёным цветом.
Дроны без группы отображаются серым цв
етом.
Кроме того, если зараженный участок установлен, он также отображается на графике. Цвет зараженного участка изменяется в зависимости от его статуса: заражённые участки красного цвета, очищенные — цвета teal. При этом выводится соответствующий текст ("Зараженная зона" или "Очищенная зона"). График имеет ограничения по оси X и Y, а также легенду, объясняющую цветовую кодиро вку.

spl
it_group
Этот метод разделяет все дроны на две группы: первая группа включает первую половину дронов, вторая группа — вторую половину. Группы назначаются путём изменения атрибута group у каждого дрона. Метод возвращает два списка: список дронов первой группы и список дронов второй  группы.

search_in
fected_area
Метод итерируется по всем дронам в группе и проверяет, находится ли каждый дрон внутри заражённого участка. Если дрон находится внутри заражённого участка, записывается соответствующее лог-сообщение об обнаружении зараженног о участка.

transm
it_information
Этот метод передаёт информацию о заражённом участке между группами дронов. Заражённый участок, установленный для одной группы, становится доступным и для другой группы, а также обновляется маршрут дронов второй группы для направления к координатам заражён ного участка.

cl
ean_infected_area
Метод фиксирует очистку заражённого участка, изменяя его статус на "очищенный" и выводя сообщение об очистке. Очистка участка обновляет его визуализ ацию на графи 
ке.

simulate_task
Этот метод реализует логику симуляции задачи поиска и очистки заражённых участков дронами. В ходе симуляции дроны первой группы ищут заражённый участок, перемещаясь случайным образом. Как только заражённый участок найден, дроны первой группы останавливаются, а дроны второй группы направляются к месту заражения для его очистки. После того как все дроны второй группы достигают заражённого участка, он считается очищенным. Визуализация позиций дронов и заражённого участка обновляется на каждом шаге симуляции.

In [ ]:
class DroneGroup:
    def __init__(self):
        self.drones = []
    
    def add_drone(self, drone):
        self.drones.append(drone)

    def check_collisions(self):
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], start=i+1):
                distance = np.linalg.norm(drone1.position - drone2.position)
                if distance < 2:  # Порог столкновения
                    drone1.points -= 10
                    drone2.points -= 10
                    drone1.log_action(f"Столкновение с Drone {drone2.drone_id}")
                    drone2.log_action(f"Столкновение с Drone {drone1.drone_id}")

    def simulate(self, steps=10):
        for step in range(steps):
            print(f"| ЭПОХА {step + 1} |")
            self.update_graph()
            for drone in self.drones:
                drone.move()
                drone.points += 1  # Начисление очков за успешный шаг без столкновения
                drone.log_action("Изменение позиции")
            self.check_collisions()  # Проверка и обработка столкновений
            self.find_best_drone()
            self.plot_group_positions()
            # После выполнения некоторых шагов симуляции
            self.update_graph()  # Обновляем граф на основе текущих позиций дронов
            self.plot_graph()  # Визуализируем граф

    def find_best_drone(self):
        best_drone = None
        max_points = -float('inf')  # Инициализируем с очень маленьким числом
        count_point = 0
        print('<=======>')
        for drone in self.drones:
            if drone.points > max_points:
                count_point +=1
                best_drone = drone
                max_points = drone.points
        print(f"Лучший дрон: Drone {best_drone.drone_id} с очками: {max_points}")
        print('<=======>')
    def update_graph(self):
        # Обновляет граф на основе текущих позиций дронов
        self.graph = Graph()  # Пересоздаем граф для обновления связей
        for i, drone1 in enumerate(self.drones):
            for j, drone2 in enumerate(self.drones[i+1:], start=i+1):
                distance = np.linalg.norm(drone1.position - drone2.position)
                if distance < 10:  # Условный порог для создания связи
                    self.graph.add_edge(drone1.drone_id, drone2.drone_id, weight=distance)
    def plot_group_positions(self):
        plt.figure(figsize=(10, 6))
        for drone in self.drones:
            if drone.group == 'first':
                plt.scatter(drone.position[0], drone.position[1], color='blue', label=f'Drone {drone.drone_id} (1st Group)')
            elif drone.group == 'second':
                plt.scatter(drone.position[0], drone.position[1], color='green', label=f'Drone {drone.drone_id} (2nd Group)')
            else:
                plt.scatter(drone.position[0], drone.position[1], color='gray', label=f'Drone {drone.drone_id} (Unassigned)')
    
        # Отображение зараженного участка (если установлен)
        if self.infected_area.is_cleaned == False:
            print("Отображаем зараженный участок")
            self.infected_area.plot()
        else:
            self.infected_area.plot()
            #print("Зараженный участок не установлен")
    
        plt.xlim(-100, 100)
        plt.ylim(-100, 100)
        plt.xlabel('X Position')
        plt.ylabel('Y Position')
        plt.title('Drone Positions by Group')
        # Создаем легенду только для уникальных элементов
        handles, labels = plt.gca().get_legend_handles_labels()
        by_label = dict(zip(labels, handles))
        plt.legend(by_label.values(), by_label.keys(), loc='upper center', bbox_to_anchor=(0.5, -0.1), shadow=True, ncol=8)
        plt.show()
    def plot_graph(self):
        G = nx.Graph()
        plt.figure(figsize=(10, 4))
        # Добавляем узлы и рёбра в граф NetworkX
        for drone_id, neighbors in self.graph.edges.items():
            G.add_node(drone_id)
            for neighbor_id, weight in neighbors.items():
                G.add_edge(drone_id, neighbor_id, weight=weight)
    
        # Используем макет Kamada-Kawai для расположения узлов
        pos = nx.kamada_kawai_layout(G)
    
        # Размеры узлов основаны на степени узла (количество связей)
        node_sizes = [50 * G.degree(n) for n in G.nodes]
    
        # Цвета рёбер основаны на их весе
        edge_colors = [G[u][v]['weight'] for u, v in G.edges]
    
        # Визуализация узлов
        nx.draw_networkx_nodes(G, pos, node_size=node_sizes, cmap=plt.cm.plasma)
    
        # Визуализация рёбер
        nx.draw_networkx_edges(G, pos, width=2, edge_color=edge_colors, edge_cmap=plt.cm.Blues)
    
        # Визуализация меток узлов
        nx.draw_networkx_labels(G, pos)
        
        plt.xlim(-1.5, 1.5)
        plt.ylim(-1.5, 1.5)
    
        plt.title("Граф взаимодействия дронов")
        mappable = plt.cm.ScalarMappable(norm=plt.Normalize(vmin=min(edge_colors), vmax=max(edge_colors)), cmap=plt.cm.Blues)
        mappable.set_array(edge_colors)
        plt.colorbar(mappable, label='Вес ребра', ax=plt.gca())
        plt.axis('off')  # выключаем отображение осей для чистоты визуализации
        plt.show()


    def split_group(self):
        midpoint = len(self.drones) // 2
        for drone in self.drones[:midpoint]:
            drone.group = 'first'
        for drone in self.drones[midpoint:]:
            drone.group = 'second'
        first_half = self.drones[:midpoint]
        second_half = self.drones[midpoint:]
        return first_half, second_half


    def search_infected_area(self):
        # Проверяем, установлена ли зараженная область
        if self.infected_area is not None:
            # Перебираем все дроны в группе и проверяем, находятся ли они внутри зараженного участка
            for drone in self.drones:
                if self.infected_area.is_inside(drone.position):
                    drone.log_action("Обнаружен зараженный участок")

    def transmit_information(self, other_group, infected_area_coords):
        # Передаем информацию о зараженном участке другой группе дронов
        other_group.infected_area = self.infected_area
        # Устанавливаем зараженный участок в текущей группе дронов
        self.infected_area = other_group.infected_area
        # Передаем информацию о зараженном участке в первую половину группы дронов
        for drone in other_group.drones:
            drone.infected_area = other_group.infected_area
        # Передаем информацию о зараженном участке во вторую половину группы дронов
        for drone in self.drones:
            drone.infected_area = self.infected_area
        for drone in other_group.drones:
            drone.route = infected_area_coords

    def clean_infected_area(self):
        # Очищаем зараженный участок
        #self.infected_area = None
        print('Участок очищен')


    def simulate_task(self, steps=100):
        found_infected = False
        for step in range(steps):
            print(f"| ЭПОХА {step + 1} |")
            first_half, second_half = self.split_group()  # Разделение на две половины
            print('Статус обнаружения зараженной зоны: ', found_infected)
            if found_infected == False:
                for drone in self.drones:
                    if drone.group == 'first':
                        drone.random_move()
                        if self.infected_area.is_inside(drone.position):
                            print(f"Drone {drone.drone_id} обнаружил зараженный участок.")
                            found_infected = True
                            for d in self.drones:
                                if d.group == 'first':  # Останавливаем дроны первой группы
                                    d.is_moving = False
                            break  # Прекращаем проверку как только зараженный участок найден
            if found_infected == True:
                
                target_x = self.infected_area.center[0] + np.sign(self.infected_area.center[0]) * 5
                target_y = self.infected_area.center[1] + np.sign(self.infected_area.center[1]) * 5
                infected_area_coords = [target_x, target_y]
                print('Координаты найденной зараженной зоны: ', infected_area_coords)
                for drone in self.drones:
                    if drone.group == 'second':  # Двигаем дроны второй группы
                        for drone in self.drones:
                            drone.log_action("Изменение позиции")
                            drone.route = [infected_area_coords]
                            drone.is_moving = True
                            drone.move_towards_target()
    
            # Проверка на очистку зараженного участка
            if found_infected:
                all_in_infected_area = all(self.infected_area.is_inside(drone.position) for drone in self.drones if drone.group == 'second')
                if all_in_infected_area:
                    print("Все дроны второй группы достигли зараженного участка. Очистка...")
                    self.infected_area.is_cleaned = True  # Обновляем статус зоны
                    self.clean_infected_area()
                    #found_infected = False
                    self.plot_group_positions()
                    break
    
            self.plot_group_positions()
            self.update_graph()








In [ ]:
# Создаем экземпляр группы дронов
drone_graph = DroneGroup()
for i in range(30):
    drone_graph.add_drone(Drone(drone_id=i, model_path="circle_3_best_drone23.pth", 
                                x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))

# Создаем зараженный участок
infected_center = [40, 40]  # Пример координат зараженного участка
infected_radius = 15  # Пример радиуса зараженного участка
drone_graph.infected_area = InfectedArea(center=infected_center, radius=infected_radius)

# Запускаем симуляцию
drone_graph.simulate_task(steps=100)

Уже на 8 эпохе первая группа дронов достигла своей цели - нашла зараженную зону, координаты зараженной зоны были переданы во вторую группу, после чего вторая группа направилась на зачистку. результат успех, все дроны из второй группы достигли зоны и очистили ее.

#### Эксперемент 2. Группа дронов 10, зараженный участок в точке 40, 40 и радиусом 15

In [ ]:
# Создаем экземпляр группы дронов
drone_graph = DroneGroup()
for i in range(10):
    drone_graph.add_drone(Drone(drone_id=i, model_path="circle_3_best_drone23.pth", 
                                x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))

# Создаем зараженный участок
infected_center = [40, 40]  # Пример координат зараженного участка
infected_radius = 15  # Пример радиуса зараженного участка
drone_graph.infected_area = InfectedArea(center=infected_center, radius=infected_radius)

# Запускаем симуляцию
drone_graph.simulate_task(steps=500)

Дроны так же нашли зараженный участок к 8 эпохе и очистили

#### Эксперемент 3. Группа дронов 50. Зараженный участок с координатами 50, 50 и радуис 10

In [ ]:
# Создаем экземпляр группы дронов
drone_graph = DroneGroup()
for i in range(50):
    drone_graph.add_drone(Drone(drone_id=i, model_path="circle_3_best_drone23.pth", 
                                x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))

# Создаем зараженный участок
infected_center = [50, 50]  # Пример координат зараженного участка
infected_radius = 10  # Пример радиуса зараженного участка
drone_graph.infected_area = InfectedArea(center=infected_center, radius=infected_radius)

# Запускаем симуляцию
drone_graph.simulate_task(steps=500)

на 22 эпохе дроны из первой группы нашли более меньший участок, и дроны из второй группы его очистили.

#### Эксперемент 4. Группа дронов 20, зараженный участок в точке 60, 60 и радиусом 7. Поскольку в этот раз размер участка крайне мал, в классе дрона требуется уменьшить скорость, с которой дроны из второй группы двигаются в сторону зараженной зоны, чтобы избежать перелетания через зону (повысить точность)

In [ ]:
class Drone:
    def __init__(self, drone_id, model_path, x=0, y=0, vx=0, vy=0, group=None):
        self.drone_id = drone_id
        self.position = np.array([x, y], dtype=np.float32) # начальная позиция [x, y]
        self.velocity = np.array([vx, vy], dtype=np.float32) # начальная скорость [Vx, Vy]
        self.points = 0  # Начальное количество очков
        self.network = AdvancedDroneNet()
        self.optimizer = optim.Adam(self.network.parameters(), lr=0.01)
        self.epsilon = 0.1  # Для эпсилон-жадной стратегии
        self.group = group  # Группа дрона: 'first' или 'second'
        self.load_model(model_path)
        self.route = [20, 25, 14, 14] # координаты, куда дроны будут перемещаться
        self.is_moving = True
    def select_action(self, state):
        if np.random.rand() < self.epsilon:  # Исследование
            return np.random.choice([0, 1, 2, 3])  # Случайный выбор действия
        else:  # Использование политики
            state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Преобразование состояния в тензор
            action_values = self.network(state_tensor)
            return torch.argmax(action_values).item()  # Выбор действия с максимальной оценкой
    def update_route(self, graph):
        # Планирование маршрута до следующей целевой точки
        current_position = (self.position[0], self.position[1])
        if self.route:
            next_target = self.route[0]  # Предположим, что route уже содержит координаты целевых точек
            planned_path = plan_route(graph, current_position, next_target)
            self.route = planned_path[1:]  # Обновляем маршрут, исключая текущую позицию
    def adapt_movement(self):
        # Адаптация движения на основе текущего маршрута и ситуации вокруг
        if self.route:
            next_step = self.route[0]
            self.velocity += acceleration_change
            self.position += self.velocity
    def move(self):
        if not self.is_moving:  # Если дрон не должен двигаться, пропускаем шаг
            return
        state = np.concatenate((self.position, self.velocity))
        #action = self.select_action(state)
        action = 2
        # Преобразование действия в изменение скорости
        acceleration_change = self.action_to_acceleration(action)
        self.velocity += acceleration_change
        self.position += self.velocity
        
        # Ограничение движения в пределах заданного пространства
        self.position = np.clip(self.position, -100, 100)
        self.velocity *= np.where((self.position == 100) | (self.position == -100), -1, 1)
    def action_to_acceleration(self, action):
        # Пример простой реализации
        if action == 0: return np.array([0.1, 0])  # Ускорение вправо
        elif action == 1: return np.array([-0.1, 0])  # Ускорение влево
        elif action == 2: return np.array([0, 0.1])  # Ускорение вверх
        elif action == 3: return np.array([0, -0.1])  # Ускорение вниз
        return np.array([0, 0])

    def log_action(self, action):
        print(f"Drone {self.drone_id}: {action}, Позиция: {self.position}, Скорость: {self.velocity}, Очки: {self.points}")
    
    def save_model(self, file_name):
        torch.save(self.network.state_dict(), file_name)
        print(f"Модель сохранена в {file_name}")

    def load_model(self, file_name):
        if file_name == 'none':
            pass
        else:
            self.network.load_state_dict(torch.load(file_name))
            self.network.eval()  # Переключение в режим оценки
            print(f"Модель загружена из {file_name}")
    def move_towards_target(self):
        if not self.route or not self.is_moving:  # Если маршрут не задан или движение остановлено
            return

        target_position = np.array(self.route[0])  # Предполагаем, что цель — первый элемент маршрута
        direction_to_target = target_position - self.position

        # Проверяем, находимся ли мы в непосредственной близости от цели (например, в радиусе 1 единицы)
        if np.linalg.norm(direction_to_target) < 1.0:
            self.is_moving = False  # Останавливаем движение дрона
            return

        # Если мы не достигли цели, продолжаем движение
        normalized_direction = direction_to_target / np.linalg.norm(direction_to_target) if np.linalg.norm(direction_to_target) else np.zeros_like(direction_to_target)
        print(normalized_direction)
        self.velocity = normalized_direction * 0.1  # Примерная скорость, направленная к цели
        self.position += self.velocity
        self.position = np.clip(self.position, -100, 100)  # Ограничиваем движение в пределах заданного пространства
       
    #случайные блуждания для поиска зараженного участка первой группы дронов.
    def action_to_acceleration_search(self, action):
        # Пример простой реализации
        if action == 0: return np.array([7, 0])  # Ускорение вправо
        elif action == 1: return np.array([-7, 0])  # Ускорение влево
        elif action == 2: return np.array([0, 7])  # Ускорение вверх
        elif action == 3: return np.array([0, -7])  # Ускорение вниз
        return np.array([0, 0])
    def random_move(self):
        action = np.random.choice([0, 1, 2, 3])
        acceleration_change = self.action_to_acceleration_search(action)
        self.velocity = acceleration_change  # Применяем изменение скорости напрямую
        self.position += self.velocity
        self.position = np.clip(self.position, -100, 100)

In [ ]:
# Создаем экземпляр группы дронов
drone_graph = DroneGroup()
for i in range(20):
    drone_graph.add_drone(Drone(drone_id=i, model_path="circle_3_best_drone23.pth", 
                                x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))

# Создаем зараженный участок
infected_center = [60, 60]  # Пример координат зараженного участка
infected_radius = 7  # Пример радиуса зараженного участка
drone_graph.infected_area = InfectedArea(center=infected_center, radius=infected_radius)

# Запускаем симуляцию
drone_graph.simulate_task(steps=1000)

На эпохе ЭПОХА 482 участок был обнаружен, и дроны его успешно очистили

#### Эксперемент 5. Группа дронов 100, зараженный участок в точке 80, 80 и радиусом 10. 

In [ ]:
class Drone:
    def __init__(self, drone_id, model_path, x=0, y=0, vx=0, vy=0, group=None):
        self.drone_id = drone_id
        self.position = np.array([x, y], dtype=np.float32) # начальная позиция [x, y]
        self.velocity = np.array([vx, vy], dtype=np.float32) # начальная скорость [Vx, Vy]
        self.points = 0  # Начальное количество очков
        self.network = AdvancedDroneNet()
        self.optimizer = optim.Adam(self.network.parameters(), lr=0.01)
        self.epsilon = 0.1  # Для эпсилон-жадной стратегии
        self.group = group  # Группа дрона: 'first' или 'second'
        self.load_model(model_path)
        self.route = [20, 25, 14, 14] # координаты, куда дроны будут перемещаться
        self.is_moving = True
    def select_action(self, state):
        if np.random.rand() < self.epsilon:  # Исследование
            return np.random.choice([0, 1, 2, 3])  # Случайный выбор действия
        else:  # Использование политики
            state_tensor = torch.FloatTensor(state).unsqueeze(0)  # Преобразование состояния в тензор
            action_values = self.network(state_tensor)
            return torch.argmax(action_values).item()  # Выбор действия с максимальной оценкой
    def update_route(self, graph):
        # Планирование маршрута до следующей целевой точки
        current_position = (self.position[0], self.position[1])
        if self.route:
            next_target = self.route[0]  # Предположим, что route уже содержит координаты целевых точек
            planned_path = plan_route(graph, current_position, next_target)
            self.route = planned_path[1:]  # Обновляем маршрут, исключая текущую позицию
    def adapt_movement(self):
        # Адаптация движения на основе текущего маршрута и ситуации вокруг
        if self.route:
            next_step = self.route[0]
            self.velocity += acceleration_change
            self.position += self.velocity
    def move(self):
        if not self.is_moving:  # Если дрон не должен двигаться, пропускаем шаг
            return
        state = np.concatenate((self.position, self.velocity))
        #action = self.select_action(state)
        action = 2
        # Преобразование действия в изменение скорости
        acceleration_change = self.action_to_acceleration(action)
        self.velocity += acceleration_change
        self.position += self.velocity
        
        # Ограничение движения в пределах заданного пространства
        self.position = np.clip(self.position, -100, 100)
        self.velocity *= np.where((self.position == 100) | (self.position == -100), -1, 1)
    def action_to_acceleration(self, action):
        # Пример простой реализации
        if action == 0: return np.array([0.1, 0])  # Ускорение вправо
        elif action == 1: return np.array([-0.1, 0])  # Ускорение влево
        elif action == 2: return np.array([0, 0.1])  # Ускорение вверх
        elif action == 3: return np.array([0, -0.1])  # Ускорение вниз
        return np.array([0, 0])

    def log_action(self, action):
        print(f"Drone {self.drone_id}: {action}, Позиция: {self.position}, Скорость: {self.velocity}, Очки: {self.points}")
    
    def save_model(self, file_name):
        torch.save(self.network.state_dict(), file_name)
        print(f"Модель сохранена в {file_name}")

    def load_model(self, file_name):
        if file_name == 'none':
            pass
        else:
            self.network.load_state_dict(torch.load(file_name))
            self.network.eval()  # Переключение в режим оценки
            print(f"Модель загружена из {file_name}")
    def move_towards_target(self):
        if not self.route or not self.is_moving:  # Если маршрут не задан или движение остановлено
            return

        target_position = np.array(self.route[0])  # Предполагаем, что цель — первый элемент маршрута
        direction_to_target = target_position - self.position

        # Проверяем, находимся ли мы в непосредственной близости от цели (например, в радиусе 1 единицы)
        if np.linalg.norm(direction_to_target) < 1.0:
            self.is_moving = False  # Останавливаем движение дрона
            return

        # Если мы не достигли цели, продолжаем движение
        normalized_direction = direction_to_target / np.linalg.norm(direction_to_target) if np.linalg.norm(direction_to_target) else np.zeros_like(direction_to_target)
        print(normalized_direction)
        self.velocity = normalized_direction * 2.0  # Примерная скорость, направленная к цели
        self.position += self.velocity
        self.position = np.clip(self.position, -100, 100)  # Ограничиваем движение в пределах заданного пространства
       
    #случайные блуждания для поиска зараженного участка первой группы дронов.
    def action_to_acceleration_search(self, action):
        # Пример простой реализации
        if action == 0: return np.array([7, 0])  # Ускорение вправо
        elif action == 1: return np.array([-7, 0])  # Ускорение влево
        elif action == 2: return np.array([0, 7])  # Ускорение вверх
        elif action == 3: return np.array([0, -7])  # Ускорение вниз
        return np.array([0, 0])
    def random_move(self):
        action = np.random.choice([0, 1, 2, 3])
        acceleration_change = self.action_to_acceleration_search(action)
        self.velocity = acceleration_change  # Применяем изменение скорости напрямую
        self.position += self.velocity
        self.position = np.clip(self.position, -100, 100)

In [ ]:
# Создаем экземпляр группы дронов
drone_graph = DroneGroup()
for i in range(100):
    drone_graph.add_drone(Drone(drone_id=i, model_path="circle_3_best_drone23.pth", 
                                x=np.random.uniform(-30, 30), y=np.random.uniform(-30, 30),
                                vx=np.random.uniform(-1, 1), vy=np.random.uniform(-1, 1)))

# Создаем зараженный участок
infected_center = [80, 80]  # Пример координат зараженного участка
infected_radius = 10  # Пример радиуса зараженного участка
drone_graph.infected_area = InfectedArea(center=infected_center, radius=infected_radius)

# Запускаем симуляцию
drone_graph.simulate_task(steps=250)

на ЭПОХА 47 зараженный участок был обнаружен, после чего все дроны из второй группы устремились его очистить и очистили
